In [2]:
import skimage.io as io
import numpy as np
import scipy
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from skimage.filters import sobel_h, sobel, sobel_v, roberts, prewitt
from commonfunctions import *
from pip._internal import main as install
from pylab import imshow, gray, show 
import cv2
from math import pi
from scipy.ndimage import interpolation as inter
from statistics import mode,variance
from skimage.measure import find_contours
from skimage.filters import threshold_otsu

In [3]:
def showBinaryImg(img):
    cv2.imshow("Images",img*255)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()
def showImg(img):
    cv2.imshow("Images",img)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

In [4]:
def customErosion(img,ST):
    countOnes= sum(sum(ST))
    imgConvolved = convolve2d(img,ST,mode="same")
    imgEroded = np.where(imgConvolved == countOnes,0,1)
    return imgEroded

def customDilation(img,ST):
    imgConvolved = convolve2d(img,ST,mode="same")
    imgDilated = np.where(imgConvolved >= 1,0,1)
    return imgDilated

In [5]:
def deskew(img):
    initAngle = 45
    angles = np.arange(-initAngle, initAngle + 1, 1)
    scoreArray = np.zeros(92)
    for i in angles:
        data = inter.rotate(img, i, reshape=False, order=0)
        sumRow = np.sum(data, axis=1)
        score = np.sum((sumRow[1:] - sumRow[:-1]) ** 2)
        scoreArray[i+45] = score

    data = inter.rotate(img, 90, reshape=True, order=0)
    sumRow90 = np.sum(data, axis=1)
    score90 = np.sum((sumRow90[1:] - sumRow90[:-1]) ** 2)
    scoreArray[91] = score90

    scoreArray = np.array(scoreArray)
    trueAngle = np.where(scoreArray == max(scoreArray))[0][0] - 45
    if(trueAngle == 46):
        rotated = inter.rotate(img, 90, reshape=True, order=0)
    else:
        rotated = inter.rotate(img, trueAngle, reshape=True, order=0)
    return rotated

In [6]:
#Binarization
import numpy as np

from skimage.util.shape import view_as_windows

def feng_threshold(img, w_size1=15, w_size2=30,
                   k1=0.15, k2=0.01, alpha1=0.1):
    """ Runs the Feng's thresholding algorithm.
    Reference:
    Algorithm proposed in: Meng-Ling Feng and Yap-Peng Tan, “Contrast adaptive
    thresholding of low quality document images”, IEICE Electron. Express,
    Vol. 1, No. 16, pp.501-506, (2004).
    Modifications: Using integral images to compute the local mean and the
    standard deviation
    @param img: The input image. Must be a gray scale image
    @type img: ndarray
    @param w_size1: The size of the primary local window to compute
        each pixel threshold. Should be an odd window
    @type w_size1: int
    @param w_size2: The size of the secondary local window to compute
        the dynamic range standard deviation. Should be an odd window
    @type w_size2: int
    @param k1: Parameter value that lies in the interval [0.15, 0.25].
    @type k1: float
    @param k2: Parameter value that lies in the interval [0.01, 0.05].
    @type k2: float
    @param alpha1: Parameter value that lies in the interval [0.15, 0.25].
    @type alpha1: float
    @return: The estimated local threshold for each pixel
    @rtype: ndarray
    """
    # Obtaining rows and cols
    rows, cols = img.shape
    i_rows, i_cols = rows + 1, cols + 1

    # Computing integral images
    # Leaving first row and column in zero for convenience
    integ = np.zeros((i_rows, i_cols), np.float)
    sqr_integral = np.zeros((i_rows, i_cols), np.float)

    integ[1:, 1:] = np.cumsum(np.cumsum(img.astype(np.float), axis=0), axis=1)
    sqr_img = np.square(img.astype(np.float))
    sqr_integral[1:, 1:] = np.cumsum(np.cumsum(sqr_img, axis=0), axis=1)

    # Defining grid
    x, y = np.meshgrid(np.arange(1, i_cols), np.arange(1, i_rows))

    # Obtaining local coordinates
    hw_size = w_size1 // 2
    x1 = (x - hw_size).clip(1, cols)
    x2 = (x + hw_size).clip(1, cols)
    y1 = (y - hw_size).clip(1, rows)
    y2 = (y + hw_size).clip(1, rows)

    # Obtaining local areas size
    l_size = (y2 - y1 + 1) * (x2 - x1 + 1)

    # Computing sums
    sums = (integ[y2, x2] - integ[y2, x1 - 1] -
            integ[y1 - 1, x2] + integ[y1 - 1, x1 - 1])
    sqr_sums = (sqr_integral[y2, x2] - sqr_integral[y2, x1 - 1] -
                sqr_integral[y1 - 1, x2] + sqr_integral[y1 - 1, x1 - 1])

    # Computing local means
    means = sums / l_size

    # Computing local standard deviation
    stds = np.sqrt(sqr_sums / l_size - np.square(means))

    # Obtaining windows
    padded_img = np.ones((rows + w_size1 - 1, cols + w_size1 - 1)) * np.nan
    padded_img[hw_size: -hw_size, hw_size: -hw_size] = img

    winds = view_as_windows(padded_img, (w_size1, w_size1))

    # Obtaining maximums and minimums
    mins = np.nanmin(winds, axis=(2, 3))

    # Obtaining local coordinates for std range calculations
    hw_size = w_size2 // 2
    x1 = (x - hw_size).clip(1, cols)
    x2 = (x + hw_size).clip(1, cols)
    y1 = (y - hw_size).clip(1, rows)
    y2 = (y + hw_size).clip(1, rows)

    # Obtaining local areas size
    l_size = (y2 - y1 + 2) * (x2 - x1 + 2)

    # Computing sums
    sums = (integ[y2, x2] - integ[y2, x1 - 1] -
            integ[y1 - 1, x2] + integ[y1 - 1, x1 - 1])
    sqr_sums = (sqr_integral[y2, x2] - sqr_integral[y2, x1 - 1] -
                sqr_integral[y1 - 1, x2] + sqr_integral[y1 - 1, x1 - 1])

    # Computing local means2
    means2 = sums / l_size

    # Computing standard deviation range
    std_ranges = np.sqrt(sqr_sums / l_size - np.square(means2))

    # Computing normalized standard deviations and extra alpha parameters
    n_stds = stds / std_ranges
    n_sqr_std = np.square(n_stds)
    alpha2 = k1 * n_sqr_std
    alpha3 = k2 * n_sqr_std

    thresholds = ((1 - alpha1) * means + alpha2 * n_stds
                  * (means - mins) + alpha3 * mins)
    for x in range(img.shape[0]):
        for y in range(img.shape[1]):
            if(img[x][y]<thresholds[x][y]):
                img[x][y]=0
            else:
                img[x][y]=1
    return img


In [7]:
#Smoothing
def signaltonoise(a, axis=None, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=0)
    return np.where(sd == 0, 0, m/sd)
def smooth(gray_img):   
    snr = signaltonoise(gray_img)
    if(snr<1):
        snr=1
    if(snr>5):
        snr=5
    sigma=(-50/4)*(snr-1)+60
    smImage=cv2.bilateralFilter(gray_img,8,int(sigma),int(sigma))
    return smImage

In [8]:
#Illumenation
def simplestColorBalance(img,s):
    s/=100
    n,m=img.shape
    f=img.flatten()
    f.sort()
    minT=f[int(s*n*m)]
    maxT=f[ n*m - int(s*n*m) - 1]
    img=np.where(img<minT,0,img)
    img=np.where(img>maxT,255,img)
    img=np.where((img!=0)&(img!=255),255*((img-minT)/(maxT-minT)),img)
    return img
def poisonScreening(img,L):
    n,m=img.shape
    img_freq=cv2.dft(img)
    for i in range(n):
        for j in range(m):
            coef=(pi*pi*i*i)/(n*n) + (pi*pi*j*j)/(m*m)
            img_freq[i][j]=(img_freq[i][j]*coef)/(L+coef)
    img=cv2.idft(img_freq)
    return img
def applyPoison(img,s,L):
    img=simplestColorBalance(img,s)
    cv2.imwrite('Balanced.png',img)
    img=poisonScreening(img,L)
    cv2.imwrite('poison.png',img)
    img=simplestColorBalance(img,s)
    cv2.imwrite('final.png',img)
    return img


In [9]:
#prespecive correction
def prespectiveCorrection(RotatedImage):
    #rotated image is the image returned from the deskew directly
    RotatedImage = RotatedImage*255
    rows,cols = RotatedImage.shape
    edges = cv2.Canny(RotatedImage,50,150,apertureSize = 3)
    
    maxLineGap = 10
    minLineLength = 60
    prespectiveSlopeThreshold = 0.05
    flag=False
    
    
    lines = cv2.HoughLinesP(edges,1,np.pi/180,15,minLineLength=minLineLength,maxLineGap=maxLineGap)
    binary = np.zeros((rows,cols)).astype('uint8')
    
    s = 0
    countLines = 0
    countOfVerticalAndhorizontalLines = 0
    for x in range(0, len(lines)):
        for x1,y1,x2,y2 in lines[x]:
            cv2.line(binary,(x1,y1),(x2,y2),255,2)
            m = ((y2-y1)/(x2-x1))
            if(abs(m)==np.inf or m==0):
                countOfVerticalAndhorizontalLines+=1
            if(abs(m) != np.inf):
                countLines+=1
                s+=m
                
    avrgSlope = s/countLines 
    percentage = (countOfVerticalAndhorizontalLines/len(lines))*100
              

    if(abs(avrgSlope)>prespectiveSlopeThreshold and percentage<80):
        
        SE = np.ones([50,50])      
        closedImage = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, SE)
        
    
        contours, hierarchy = cv2.findContours(closedImage,  cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        cnts = sorted(contours, key = cv2.contourArea, reverse = True)[:5]
    
        
        for box in cnts:
            epsilon = 0.02*cv2.arcLength(box,True)
            approx = cv2.approxPolyDP(box,epsilon,True) 
            for i in range(0,len(approx)):
                RotatedImage = cv2.circle(RotatedImage, (approx[i,0,0],approx[i,0,1]), 2, 200, 2)
            if(len(approx)==4):
                flag=True
                points=approx
                break
            
        if flag==True:
            
            margin = 20
            
            points[0,0,0]+=margin
            points[0,0,1]-=margin
            points[1]-=margin
            points[2,0,0]-=margin
            points[2,0,1]+=margin
            points[3]+=margin
            
            
            cv2.drawContours(RotatedImage, [points], -1, (0, 255, 0), 2)
            
            rect = np.zeros((4, 2), dtype = "float32")
            s1 = points.sum(axis = 1)
            s = s1.sum(axis=1)
            
            
            rect[0] = s1[np.argmin(s)]
            rect[2] = s1[np.argmax(s)]
            diff = np.diff(s1, axis = 1)
            rect[1] = s1[np.argmin(diff)]
            rect[3] = s1[np.argmax(diff)]
            
            (tl, tr, br, bl) = rect
            widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
            widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
            maxWidth = max(int(widthA), int(widthB))
            heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
            heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
            maxHeight = max(int(heightA), int(heightB))
            
            dst = np.array([
            		[0, 0],
            		[maxWidth - 1, 0],
            		[maxWidth - 1, maxHeight - 1],
            		[0, maxHeight - 1]], dtype = "float32")
                
        
            M = cv2.getPerspectiveTransform(rect, dst)
            warped = cv2.warpPerspective(RotatedImage, M, (maxWidth, maxHeight)) 
            
        else:warped = RotatedImage
            
    else:
        warped = RotatedImage
        
    return warped

In [10]:
#determine if the image needs illumination evening and if it need feng thresholding 
#if image has uneven illumination => use poisson
#if image has uneven illumination or a very low contrast(another function not done yet) => use feng
#Return True if image is good and false if image is uneven
def imageState(imgGray):
    blurred = cv2.GaussianBlur(imgGray, (25, 25), 0)
    no_text = imgGray * ((imgGray/blurred)>0.99)                
    no_text[no_text<10] = no_text[no_text>20].mean()      
    no_bright = no_text.copy()
    no_bright[no_bright>220] = no_bright[no_bright<220].mean()
    std = no_bright.std()
    bright = (no_text>220).sum()
    if std>18 or (no_text.mean()<200 and bright>8000):
        return False
    else:
        return True


In [11]:
img= cv2.imread("02.PNG")
countourImg=np.copy(img)
gray_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
isEven = imageState(gray_img)
IlluminatedImage=gray_img
if(not isEven):
    IlluminatedImage=applyPoison(gray_img,0.1,0.1)
    IlluminatedImage=IlluminatedImage.astype(np.uint8)
smoothedImage=smooth(IlluminatedImage)
if  not isEven:
    BinarizedImage=feng_threshold(smoothedImage)
else:
    t =threshold_otsu(smoothedImage)
    BinarizedImage=np.where(smoothedImage>t,1,0)
BinarizedImage=1-BinarizedImage
RotatedImage=deskew(BinarizedImage)
#correctedPrespective=prespectiveCorrection(RotatedImage)
BinarizedImage=1-RotatedImage
cv2.imwrite("Binarized.png",BinarizedImage*255)


True

In [12]:
#divide binarized image into segments
#get Horizontal Projection 
imgCopy= np.copy(BinarizedImage)
img=imgCopy
img=1-img
def rowProjection(img):
    proj = np.sum(img,1)
    maxProjection = np.max(proj) 
    result = np.zeros(img.shape) 
    # Draw a line for each row 
    for row in range(img.shape[0]):
        result[row,0:int(proj[row])]=1  
    result=1-result
    cv2.imwrite("result.png",result*255)
    return maxProjection,result

In [13]:
#estimate staff position 
def calcStaffPos(rowHist,maxProjection,thr):
    n,m=rowHist.shape
    thres=int(maxProjection*thr)
    start=end=0
    width=[]
    peaksMid=[]
    for i in range(n-1):
        if(rowHist[i][thres]!=rowHist[i+1][thres]):
            if rowHist[i][thres]==1:
                #end of zeros
                start=i
            else:
                #end of ones
                end=i+1
                peaksMid.append(int((start+end)/2))
                width.append((end-start))
    return np.array(width),np.array(peaksMid)

#estimate staff segment position 
#TODO: try to make segments into blocks with ST
def calcSegmentPos(rowHist):
    n,m=rowHist.shape
    thres=20
    start=end=0
    width=[]
    peaksMid=[]
    for i in range(n-1):
        if(rowHist[i][thres]!=rowHist[i+1][thres]):
            if rowHist[i][thres]==1:
                #end of zeros
                start=i
            else:
                #end of ones
                end=i+1
                peaksMid.append(int((start+end)/2))
                width.append((end-start))
    return np.array(width),np.array(peaksMid)

def filterPeaks(peaksMids,widths):
    if(len(peaksMids)<=1):
        return widths,peaksMids
    newPeaks=[]
    newWidth=[]
    sumW=0
    for i in widths:
        sumW+=i
    avgW=sumW/len(widths)
    #if i dont have flase peaks for clusters then this will result in a problem
    var = variance(widths)
    for i in range(len(peaksMids)):
        if(widths[i]>=avgW or (var<1000 and (avgW-widths[i])<20)):
            newPeaks.append(peaksMids[i])
            newWidth.append(widths[i])
    return newWidth,newPeaks

#TODO:add validation so Idont get out of bound error
#divide image into segments
def imgStaffSegments(img,segMids,widths,staffS):
    segments=[]
    staffS=int(staffS)
    n,m=img.shape
    for i in range(len(segMids)):
        if(segMids[i]-int(widths[i]/2)-2*staffS < 0):
            segments.append(img[0:segMids[i]+int(widths[i]/2)+2*staffS,:])
        elif(segMids[i]+int(widths[i]/2)+2*staffS>m):
            segments.append(img[segMids[i]-int(widths[i]/2)-2*staffS:m,:])
        else:
            segments.append(img[segMids[i]-int(widths[i]/2)-2*staffS:segMids[i]+int(widths[i]/2)+2*staffS,:])
    return segments

def maxStaffSpace(peaksMids,width):
    maxSpace=0
    for i in range(len(peaksMids)):
        if(i%5<4 and i+1<len(peaksMids)and peaksMids[i+1]-peaksMids[i]>maxSpace):
            maxSpace=peaksMids[i+1]-peaksMids[i]
            maxSpace=maxSpace-(width[i]/2+width[i+1]/2)
    return maxSpace



In [14]:

#divide image into segments
#estimate max staff space
img = 1 - np.copy(BinarizedImage)

maxProjection,result=rowProjection(img)
staffWidth,peaksMids=calcStaffPos(result,maxProjection,0.6)
maxSpace=maxStaffSpace(peaksMids,staffWidth)

#estimate staff segment position

segWidth,segMids=calcSegmentPos(result)

#filter peaks
segWidth,segMids=filterPeaks(segMids,segWidth)


#divide image into segments and divide image => return array of images each image has staff segment

imgSegments = imgStaffSegments(img,segMids,segWidth,maxSpace)

imgindex=0
for i in imgSegments:
    cv2.imwrite("segments/"+str(imgindex)+".png",i*255)
    imgindex+=1

In [15]:
#loop in array of images and remove staff from each
def removeStaffRow(imgOriginal,midPoint,curWidth):
    thresPixel=curWidth
    for i in range(imgOriginal.shape[1]):
        pixelSum= sum(imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1])
        if(pixelSum<=thresPixel):
           # print(imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1])
            imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1]=0
    return imgOriginal

#remove staff from array of imgs of segments
imgsStaffRemoved=[]
for simg in imgSegments:
    # simg=1-simg
    maxProjection,result=rowProjection(simg)
    staffWidth,peaksMids=calcStaffPos(result,maxProjection,0.6)
    simg=simg.astype(np.float)

    for i in range(len(peaksMids)):
        simg=removeStaffRow(simg,peaksMids[i],staffWidth[i])
    ST=np.ones((2,1))
    simg=cv2.dilate(simg,ST)
    simg=1-simg
    imgsStaffRemoved.append(simg)


imgindex=0
for i in imgsStaffRemoved:
    cv2.imwrite("staffRemoved/"+str(imgindex)+".png",i*255)
    imgindex+=1
#return array of images without staff

In [16]:
#discard smaller contours overlapped with another bigger contours
#reorder contours from left to right and from top to dowm
def checkOverlapped(c,imgContours):
    cXmin,cXmax,cYmin,cYmax = c
    for Xmin,Xmax,Ymin,Ymax in imgContours:
        if(cXmin > Xmin and cXmax < Xmax and cYmin > Ymin and cYmax < Ymax):
            return True
    return False
def filterContours(imgContours):
    filtered=[]
    for c in imgContours:
        if(not checkOverlapped(c,imgContours)):
            filtered.append(c)
    filtered.sort()
    return filtered



In [ ]:
# if color is white => symbol , black => background
# another matrix => visited pixels

#attempt algorithm to find corner 
#1.find suspected point as a begining of corner
#2.iterate from the right pixel
#3.if path from this pixel is longer than 4 and dominating direction is DOWN AND RIGHT and does not form a loop(closed path)
#  then this is a corner=> mark as visited
#4.repeat untill all white pixels are done

#return path length and top two dominating directions
def checkPathRightDown(yR,xR,yD,xD,visited,isCorner):
    #if valid path update visited and isCorner list
    h,w=visited.shape
    if(visited[yR][xR]):
        return
    #check right path
    nextX=xR
    nextY=yR
    pathR=[]
    pathR.append([nextX,nextY])
    while(nextX+1 != w and nextY+1 != h):
        #check right down
        if(skeleton[nextY+1][nextX+1]):
            nextX=nextX+1
            nextY=nextY+1
            pathR.append([nextX,nextY])
        #check right pixel
        elif(skeleton[nextY][nextX+1]):
            nextX=nextX+1
            pathR.append([nextX,nextY])
        #check down
        elif(skeleton[nextY+1][nextX]):
            nextY=nextY+1
            pathR.append([nextX,nextY])
        #if none exit loop
        else:
            break
    #check if path down is long enough countD>=5
    nextX=xD
    nextY=yD
    countD=0
    while(nextX+1 != w and nextY+1 != h and nextX !=0):
        #check down
        if(skeleton[nextY+1][nextX]):
            nextY=nextY+1
            countD+=1
        #check right down
        elif(skeleton[nextY+1][nextX+1]):
            nextX=nextX+1
            nextY=nextY+1
        #check left down
        elif(skeleton[nextY+1][nextX-1]):
            nextX=nextX-1
            nextY=nextY+1
        #if none exit loop
        else:
            break
    if(len(pathR)>=5 and countD>=4):
        isCorner.append([xD,yD])
        for i in pathR:
            pX=i[0]
            pY=i[1]
            visited[pY][pX]=1

def findVinverted(skeleton):
    h,w=skeleton.shape
    visited = np.zeros((h,w))
    isCorner=[]
    index=0
    pixelIndices = np.where(skeleton==1)
    pixelCount =len(pixelIndices[0])
    for i in range(pixelCount):
        y = pixelIndices[0][i]
        x = pixelIndices[1][i]
        #check if in white pixel is on borders
        if(x+1 == w or y+1 == h or x== 0 or y==0):
            continue
        downR=skeleton[y+1][x+1]
        down=skeleton[y+1][x]
        right=skeleton[y][x+1]
        downL=skeleton[y+1][x-1]
        if (downR and downL):
            checkPathRightDown(y+1,x+1,y+1,x-1,visited,isCorner)
        elif (downR and down):
            checkPathRightDown(y+1,x+1,y+1,x,visited,isCorner)
        elif (right and downL):
            checkPathRightDown(y,x+1,y+1,x-1,visited,isCorner)
        elif (right and down):
            checkPathRightDown(y,x+1,y+1,x,visited,isCorner)
    return len(isCorner),visited


# if color is white => symbol , black => background
# another matrix => visited pixels

#attempt algorithm to find corner 
#1.find suspected point as a begining of corner
#2.iterate from the right pixel
#3.if path from this pixel is longer than 4 and dominating direction is Up AND RIGHT and does not form a loop(closed path)
#  then this is a corner=> mark as visited
#4.repeat untill all white pixels are done

#return path length and top two dominating directions
def checkPathRightUp(yR,xR,yU,xU,visited,isCorner):
    #if valid path update visited and isCorner list
    h,w=visited.shape
    if(visited[yR][xR]):
        return
    nextX=xR
    nextY=yR
    pathR=[]
    pathR.append([nextX,nextY])
    count=0
    while(nextX+1 != w and nextY != 0):
        #check right pixel
        if(skeleton[nextY][nextX+1]):
            nextX=nextX+1
            pathR.append([nextX,nextY])
        #check right up
        elif(skeleton[nextY-1][nextX+1]):
            nextX=nextX+1
            nextY=nextY-1
            pathR.append([nextX,nextY])
        #check up
        elif(skeleton[nextY-1][nextX]):
            nextY=nextY-1
            pathR.append([nextX,nextY])
        #if none exit loop
        else:
            break
    nextX=xU
    nextY=yU
    countU=0
    while(nextX+1 != w and nextY != 0 and nextX != 0):
        #check up
        if(skeleton[nextY-1][nextX]):
            nextY=nextY-1
            countU+=1
        #check left up pixel
        elif(skeleton[nextY-1][nextX-1]):
            nextX=nextX-1
            nextY=nextY-1
        #check right up
        elif(skeleton[nextY-1][nextX+1]):
            nextX=nextX+1
            nextY=nextY-1
        #if none exit loop
        else:
            break
    if(len(pathR)>=5 and countU>=4):
        isCorner.append([xR,yR])
        for i in pathR:
            pX=i[0]
            pY=i[1]
            visited[pY][pX]=1

def findV(skeleton):
    h,w=skeleton.shape
    visited = np.zeros((h,w))
    isCorner=[]
    index=0
    pixelIndices = np.where(skeleton==1)
    pixelCount =len(pixelIndices[0])
    for i in range(pixelCount):
        y = pixelIndices[0][i]
        x = pixelIndices[1][i]
        #check if in white pixel is on borders
        if(x+1 == w or y+1 == h or x== 0 or y==0):
            continue
        upR=skeleton[y-1][x+1]
        up=skeleton[y-1][x]
        right=skeleton[y][x+1]
        upL=skeleton[y-1][x-1]
        if (upR and upL):
            checkPathRightUp(y-1,x+1,y-1,x-1,visited,isCorner)
        elif (upR and up):
            checkPathRightUp(y-1,x+1,y-1,x,visited,isCorner)
        elif (right and up):
            checkPathRightUp(y,x+1,y-1,x,visited,isCorner)
        elif (right and upL):
            checkPathRightUp(y,x+1,y-1,x-1,visited,isCorner)
    return len(isCorner),visited



In [17]:
#convert image into contours
imgindex=0
segContours=[]
for seg in imgsStaffRemoved:
    contours = find_contours(seg, 0.8)
    imgContours=[]
    imgSymbols=[]
    index=0
    for contour in contours:
        x = contour[:,1]
        y = contour[:,0]
        [Xmin, Xmax, Ymin, Ymax] = [np.amin(x), np.amax(x), np.amin(y), np.amax(y)]
        imgContours.append([Xmin, Xmax, Ymin, Ymax])

    imgContours = filterContours(imgContours)

    for Xmin,Xmax,Ymin,Ymax in imgContours:
        imgSymbol=seg[int(Ymin):int(Ymax+1),int(Xmin):int(Xmax+1)]
        ST=np.ones((11))
        imgSymbol = 1-imgSymbol
        imgSymbol=cv2.dilate(imgSymbol,ST)
        imgSymbol = 1-imgSymbol
        imgSymbols.append(imgSymbol)
        cv2.imwrite("contours/"+str(imgindex)+"_"+str(index)+".png",imgSymbol*255)
        index+=1

    imgindex+=1
    segContours.append(imgSymbols)

In [ ]:
def extractFeatures(img):
    features = []
    #no of blackblobs
    #no of white blobs
    #blobs direction (right or left of vertical line)
    #blobs up or down
    #update image to remove blobs
    #no of vertical lines
    #no of beams
    #no of up flags
    #no of down flags

    #aspect ratio??
    #chain code of skelton??
    
    return features
